In [1]:
import os
import glob
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

df = pd.read_csv('data/fr.openfoodfacts.org.products.csv', sep='\t')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320772 entries, 0 to 320771
Columns: 162 entries, code to water-hardness_100g
dtypes: float64(106), object(56)
memory usage: 396.5+ MB


In [3]:
# list columns and their types
df.dtypes

code                                           object
url                                            object
creator                                        object
created_t                                      object
created_datetime                               object
last_modified_t                                object
last_modified_datetime                         object
product_name                                   object
generic_name                                   object
quantity                                       object
packaging                                      object
packaging_tags                                 object
brands                                         object
brands_tags                                    object
categories                                     object
categories_tags                                object
categories_fr                                  object
origins                                        object
origins_tags                

In [4]:
# Importing necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

# Load your data
# df = pd.read_csv('your_data.csv')

# Drop columns with too many missing values and non-numeric columns
#df.dropna(thresh=len(df) * 0.9, axis=1, inplace=True)

#df.dtypes

In [5]:
#df = df.select_dtypes(include=['float64'])

# Separate features and target
X = df.drop('nutrition_grade_fr', axis=1)
y = df['nutrition_grade_fr']

# Use RandomForest model to evaluate feature importances
clf = RandomForestClassifier(n_jobs=-1)
clf.fit(X, y)

# Plot feature importances before selection
importances = clf.feature_importances_
features_before = X.columns
plt.figure(figsize=(10, 6))
plt.barh(features_before, importances)
plt.xlabel('Feature Importance')
plt.title('Feature Importances Before Selection')
plt.show()

# Use SelectFromModel to select important features
sfm = SelectFromModel(clf, threshold=0.05)  # Adjust the threshold as needed
sfm.fit(X, y)

# Create new DataFrame with selected features
X_important = sfm.transform(X)
selected_features = X.columns[sfm.get_support()]

# Plot feature importances after selection
clf.fit(X_important, y)
importances_after = clf.feature_importances_
plt.figure(figsize=(10, 6))
plt.barh(selected_features, importances_after)
plt.xlabel('Feature Importance')
plt.title('Feature Importances After Selection')
plt.show()


ValueError: could not convert string to float: 'http://world-fr.openfoodfacts.org/produit/0000000003087/farine-de-ble-noir-ferme-t-y-r-nao'